In [1]:
import findspark
findspark.init("/home/rodolfo/spark-3.3.1-bin-hadoop3")
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("lr").getOrCreate()

22/11/30 21:22:59 WARN Utils: Your hostname, rodolfo-300E5M-300E5L resolves to a loopback address: 127.0.1.1; using 192.168.15.11 instead (on interface wlp3s0)
22/11/30 21:22:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/30 21:23:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/30 21:23:01 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
data = spark.read.csv("../../data/Ecommerce_Customers.csv", inferSchema=True, header=True)

In [5]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [6]:
data.show(2)

+--------------------+--------------------+---------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|   Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+---------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|   Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
+--------------------+--------------------+---------+------------------+------------------+------------------+--------------------+-------------------+
only showing top 2 rows



In [7]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [9]:
assembler = VectorAssembler(
    inputCols=[
        "Avg Session Length",
        "Time on App",
        "Time on Website",
        "Length of Membership"
    ],
    outputCol="features"
)

In [10]:
output = assembler.transform(data)

In [11]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [12]:
output.select("features").show(2)

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
+--------------------+
only showing top 2 rows



In [13]:
final_data = output.select("features", "Yearly amount spent")

In [14]:
final_data.show(2)

+--------------------+-------------------+
|            features|Yearly amount spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
+--------------------+-------------------+
only showing top 2 rows



In [15]:
train, test = final_data.randomSplit([0.7, 0.3])

In [16]:
train.show(2)

+--------------------+-------------------+
|            features|Yearly amount spent|
+--------------------+-------------------+
|[29.5324289670579...|  408.6403510726275|
|[30.3931845423455...|  319.9288698031936|
+--------------------+-------------------+
only showing top 2 rows



In [17]:
test.show(2)

+--------------------+-------------------+
|            features|Yearly amount spent|
+--------------------+-------------------+
|[30.5743636841713...| 442.06441375806565|
|[30.8364326747734...|  467.5019004269896|
+--------------------+-------------------+
only showing top 2 rows



In [18]:
print("Train size ", train.count())
print("Test size ", test.count())

Train size  340
Test size  160


In [21]:
lr = LinearRegression(
    labelCol="Yearly amount spent"
).fit(train)

22/11/30 21:29:29 WARN Instrumentation: [3cd0c8a5] regParam is zero, which might cause numerical instability and overfitting.


In [23]:
test_results = lr.evaluate(test)

In [24]:
test_results.residuals.show(2)

+-------------------+
|          residuals|
+-------------------+
|0.12531027158769348|
| -5.434840871323047|
+-------------------+
only showing top 2 rows



In [25]:
test_results.rootMeanSquaredError

9.526775327841086

In [26]:
test_results.r2

0.9843840743399634